In [482]:
from data_rnn import load_ndfa, load_brackets
# from data_prep import pad_and_convert
import pandas as pd
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau
from torch.utils.tensorboard import SummaryWriter

In [483]:
#runs = 'Question_8'

In [484]:
writer = SummaryWriter()
#writer = SummaryWriter(runs)

In [485]:
x_train_ndfa, (i2w_ndfa, w2i_ndfa) = load_ndfa(n=150_000)
x_train_brackets, (i2w_brackets, w2i_brackets) = load_brackets(n=150_000)


In [486]:
print(''.join([i2w_ndfa[i] for i in x_train_ndfa[50]]))


ss


In [487]:
# print(''.join([i2w_brackets[i] for i in x_train_brackets[10_000]]))


In [488]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [489]:
class LSTM(nn.Module):
    def __init__(self, vocab_size, emb_size, h, num_char, n_layers=1):
        super(LSTM, self).__init__()

        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.lstm = nn.LSTM(input_size=emb_size, hidden_size=h, num_layers=n_layers, batch_first=True)
        self.fc = nn.Linear(h, num_char)
        # self.batch_norm = nn.BatchNorm1d(num_char)
    
    def forward(self, input_seq, h=None):
        embedded = self.embedding(input_seq)
        lstm_out, hidden = self.lstm(embedded, h)
        # lstm_out = lstm_out[:, -1, :]
        output = self.fc(lstm_out)
        # output = output.permute(0, 2, 1)
        # output = self.batch_norm(output)
        # output = output.permute(0, 2, 1)
        # raise Exception('stop')
        return output, hidden

In [490]:
def create_target2(padded_batches):
    target_batches = []

    for batch in padded_batches:
        target_batch = torch.cat((batch[:, 1:], torch.zeros(batch.size(0), 1).int()), dim=1)
        target_batches.append(target_batch)
    return target_batches

In [491]:
len(x_train_ndfa)

150000

In [492]:
def pad_and_convert3(batch, w2i, batch_size):
    start_token = w2i['.start']
    end_token = w2i['.end']
    pad_token = w2i['.pad']

    num_examples = len(batch)
    num_batches = (num_examples + batch_size - 1) // batch_size
    
    # Create batches
    batches = [batch[i * batch_size: (i + 1) * batch_size] for i in range(num_batches)]
    
    padded_batches = []
    
    for b in batches:
        # Pad each sequence in the batch to the maximum length within the batch
        b = [[start_token] + x + [end_token] for x in b]
        max_len = max(len(x) for x in b)
        padded_batch = [x + [pad_token] * (max_len - len(x)) for x in b]
        padded_batches.append(padded_batch)

    # Convert the padded batches to PyTorch tensors
    padded_batches = [torch.tensor(pb, dtype=torch.long) for pb in padded_batches]

    return padded_batches

pad_and_convert maakt erg lange sequences, moet 158 + .start + .end niet de langst zijn?

Heb een andere functie maar hij doet nog steeds dezelfde loss (0)

In [493]:
x_train_ndfa_padded3 = pad_and_convert3(x_train_ndfa, w2i_ndfa, batch_size=32)
# x_train_brackets_padded3 = pad_and_convert3(x_train_brackets, w2i_brackets, batch_size=128)

In [494]:
target_ndfa2 = create_target2(x_train_ndfa_padded3)

In [495]:
vocab_size = len(w2i_ndfa)
emb_size = 32
h = 16
num_char = vocab_size
n_layers = 1

In [496]:
model = LSTM(vocab_size=vocab_size, emb_size=emb_size, h=h, num_char=num_char, n_layers=1)

In [497]:
num_epochs = 50
learning_rate = 0.1

In [498]:
criterion = nn.CrossEntropyLoss(reduction='sum', ignore_index=0)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
# scheduler = StepLR(optimizer, step_size=10, gamma=0.1) 
# scheduler = ReduceLROnPlateau(optimizer, mode='min', patience=5, factor=0.1, verbose=True)


In [499]:
trainloader = list(zip(x_train_ndfa_padded3, target_ndfa2))

In [500]:
import torch.distributions as dist
def sample(lnprobs, temperature=1.0): 
    """
    Sample an element from a categorical distribution
    :param lnprobs: Outcome logits
    :param temperature: Sampling temperature. 1.0 follows the given
        distribution, 0.0 returns the maximum probability element. :return: The index of the sampled element.
    """
    if temperature == 0.0:
        return lnprobs.argmax()
    p = F.softmax(lnprobs / temperature, dim=0)
    cd = dist.Categorical(p)
    return cd.sample()

In [501]:
max_length = 50
for epoch in range(num_epochs):
    total_loss = 0.0
    total_tokens = 0

    for batch_idx, (inputs, targets) in enumerate(trainloader):
        # print(f'Batch Index: {batch_idx}, Batch Size: {inputs.size(0)}')
        # print(inputs.shape)
        # print('targets',targets.shape)

        # print('targets reshape', targets.shape)

        model.train()        
        optimizer.zero_grad()

        h = None
        
        output, _ = model(inputs, h)
        # print('output shape', output.shape)
        output = output.reshape(-1, vocab_size)
        targets = targets.reshape(-1)
        # print('output shape reshaped', output.shape)
        # print('target shape reshaped', targets.shape)

        loss = criterion(output, targets)  

        total_loss += loss.item()
        non_pad_tokens = torch.sum(targets != 0).item()
        total_tokens += non_pad_tokens
        loss /= non_pad_tokens

        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=2.0)

        optimizer.step()



        # print(f'Epoch [{epoch+1}/{num_epochs}], Iteration [{batch_idx+1}/{len(dataloader_ndfa2)}], Loss: {total_loss:.4f}')

    average_loss = total_loss / total_tokens

    for name, param in model.named_parameters():
        if param.grad is not None:
            writer.add_scalar(f'Gradient Norm/{name}', param.grad.norm().item(), global_step=epoch)
    
    writer.add_scalar('Training loss', average_loss, global_step=epoch)

    # scheduler.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Average Loss: {average_loss:.4f}')

torch.save(model.state_dict(), 'lstm_model.pth')

Epoch [1/50], Average Loss: 0.1448
Epoch [2/50], Average Loss: 0.1440
Epoch [3/50], Average Loss: 0.1461
Epoch [4/50], Average Loss: 0.1550
Epoch [5/50], Average Loss: 0.1466


KeyboardInterrupt: 

In [458]:
writer.close()

In [459]:
%reload_ext tensorboard

In [460]:
%tensorboard --logdir=runs --host localhost

Reusing TensorBoard on port 6007 (pid 95542), started 18:49:25 ago. (Use '!kill 95542' to kill it.)

In [49]:
%tensorboard --logdir=runs